<a href="https://colab.research.google.com/github/gt-cse-6040/skills_oh_week_05/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -q https://raw.githubusercontent.com/gt-cse-6040/skills_oh_week_05/main/setup.sh
!sh setup.sh > /dev/null 2>&1
!rm setup.sh

# Debugging midterm problems

Below are several exercises from Midterm 1 (Fall 2022) with **incorrect** solutions. The general topic of the exam is dealing with data from the Reddit API. Skills covered are processing nested data, string manipulation, and implementing basic logic in Python.

## Global imports and initial data load
This isn't key to debugging but it is necessary to run the rest of the code in the notebook.

In [ ]:
### Global Imports
import json
import re
from collections import defaultdict
with open('resource/asnlib/publicdata/learnpython_top_800_all.json') as f:
    raw_reddit_data = json.load(f)
with open('resource/asnlib/publicdata/stopwords.json') as f:
    stopwords = set([s.replace("'", "") for s in json.load(f)])

## Exercise 1 - (**2** Points): 

**Requirements**

Define the function `analyze_structure(some_object)`. The input `some_object` will either be a `list` or a `dict`. 

- For a `list` input the function should return a `dict` with the following keys: `{'type', 'len', `**`'value_type'}`**. 

- For a `dict` input the function should return a `dict` with the following keys: `{'type', 'keys', `**`'value_types'}`**.

See the charts below for more information on the key/value pairs:

**If the input is a `list`** your result should have these key/value pairs.

|key|value|type of value|examples (comma separated)|  
|---|---|---|---|  
|"type"|"list"|`str`|The string `"list"` is the only acceptable value|  
|"len"|length of `some_object`|`int`|`1`, `4`, `500`|  
|"value_type"|The type of the first element of `some_object` (cast to a `str`)|`str`|`"<class 'str'>"`, `"<class 'dict'>"`|  

**If the input is a `dict`** your result should have these key/value pairs.

|key|value|type of value|examples (comma separated)|  
|---|---|---|---|  
|"type"|"dict"|`str`|The string `"dict"` is the only acceptable value|  
|"keys"|set of `some_object`'s keys|`set`|`{'data', 'kind'}`|  
|"value_types"|`dict` mapping each of `some_object`'s keys to the type (cast to `str`) of the value associated with it.|`dict` mapping `str` keys to `str` values|`{'kind': "<class 'str'>", 'data': "<class 'dict'>"}`|  

** Note:** Don't forget to cast the `'value_type'` for `list`s and values of `'value_types'` for `dict`s  to strings!   

**Note:** You can use `str(type(x))` to cast the `type` of `x` to a `str`.

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
{'type': 'list', 'len': 8, 'value_type': "<class 'dict'>"}

{'type': 'dict', 'keys': {'kind', 'data'}, 'value_types': {'kind': "<class 'str'>", 'data': "<class 'dict'>"}}

{'type': 'dict', 'keys': {'geo_filter', 'before', 'modhash', 'after', 'dist', 'children'}, 'value_types': {'after': "<class 'str'>", 'dist': "<class 'int'>", 'modhash': "<class 'str'>", 'geo_filter': "<class 'str'>", 'children': "<class 'list'>", 'before': "<class 'NoneType'>"}}

{'type': 'list', 'len': 100, 'value_type': "<class 'dict'>"}

{'type': 'dict', 'keys': {'kind', 'data'}, 'value_types': {'kind': "<class 'str'>", 'data': "<class 'dict'>"}}
```
<!-- Include any shout outs here -->

**Note** this demo will run your solution **5** times, diving level by level into the raw data. Each individual run's output is separated by a blank line.

In [ ]:
### Exercise 1 solution
def analyze_structure(some_object):
    assert isinstance(some_object, (list, dict)), f'argument must be `list` or `dict`, {type(some_object)} was given.'
    if isinstance(some_object, list):
        # Handle the `list` case
        return {'type': 'list',
                'len': len(some_object),
                'value_type': str(type(some_object[0]))}
    elif isinstance(some_object, dict):
        # Handle the `dict` case
        return {'type': 'dict',
                'keys': set(some_object.keys()),
                'value_type': {k: str(type(v)) for k, v in some_object.items()}}
    
### demo function calls
print(analyze_structure(raw_reddit_data))
print()
print(analyze_structure(raw_reddit_data[0]))
print()
print(analyze_structure(raw_reddit_data[0]['data']))
print()
print(analyze_structure(raw_reddit_data[0]['data']['children']))
print()
print(analyze_structure(raw_reddit_data[0]['data']['children'][0]))

{'type': 'list', 'len': 8, 'value_type': "<class 'dict'>"}

{'type': 'dict', 'keys': {'data', 'kind'}, 'value_type': {'kind': "<class 'str'>", 'data': "<class 'dict'>"}}

{'type': 'dict', 'keys': {'before', 'geo_filter', 'dist', 'after', 'children', 'modhash'}, 'value_type': {'after': "<class 'str'>", 'dist': "<class 'int'>", 'modhash': "<class 'str'>", 'geo_filter': "<class 'str'>", 'children': "<class 'list'>", 'before': "<class 'NoneType'>"}}

{'type': 'list', 'len': 100, 'value_type': "<class 'dict'>"}

{'type': 'dict', 'keys': {'data', 'kind'}, 'value_type': {'kind': "<class 'str'>", 'data': "<class 'dict'>"}}


<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 1. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [ ]:
### test_cell_ex1
### BEGIN HIDDEN TESTS
import dill
import hashlib
with open('resource/asnlib/public/hash_check.pkl', 'rb') as f:
    hash_check = dill.load(f)
for fname in ['testers.py', '__init__.py', 'test_utils.py']:
    hash_check(f'tester_fw/{fname}', f'resource/asnlib/public/{fname}')
del hash_check
del dill
del hashlib
### END HIDDEN TESTS
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_1', 
    'func': analyze_structure, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'some_object':{
            'dtype':'dict', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'dict',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'S90rT5WLPFy08a82F6SXyiKoyeTi33DOHh7bxXASQw0=', path='resource/asnlib/publicdata/')
for _ in range(20):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

### BEGIN HIDDEN TESTS
tester = Tester(conf, key=b'bE_KdJGq_bhBuoTZRi37O7tu3s38ac4bJUsgUEXnK8s=', path='resource/asnlib/publicdata/encrypted/')
for _ in range(20):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise
### END HIDDEN TESTS
print('Passed! Please submit.')

5e3f0d781ba1b93fcce1788f8448f8ec
5e3f0d781ba1b93fcce1788f8448f8ec
52243648c34549fa35bbbdf59ae56057
52243648c34549fa35bbbdf59ae56057
ed83b476ee8f414c417022217cf79354
ed83b476ee8f414c417022217cf79354


AssertionError: ignored

We have an assertion error telling us that our output is incorrect. Let's drill down and see exactly where...

In [ ]:
returned_output_vars['output_0'] == true_output_vars['output_0']

False

We see again using the test case variables that the output isn't matching. Next, let's see if the returned type is the same as the expected type...

In [ ]:
isinstance(returned_output_vars['output_0'], type(true_output_vars['output_0']))

True

The types match, so our solution isn't completely off. The next step is to see what type we're dealing with. This will dictate how we continue our debugging efforts!

In [ ]:
type(true_output_vars['output_0'])

dict

We're dealing with dictionaries. When dealing with dictionaries, the first thing to check is whether the keys are the same...

In [ ]:
returned_output_vars['output_0'].keys() == true_output_vars['output_0'].keys()

False

The keys aren't the same. Let's see if there are any keys in the returned result that aren't expected...

In [ ]:
returned_output_vars['output_0'].keys() - true_output_vars['output_0'].keys()

{'value_type'}

The key 'value_type' is in our returned output but not in the true output. Let's do the reverse and see if there are any expected keys which are not in our returned output...

In [ ]:
true_output_vars['output_0'].keys() - returned_output_vars['output_0'].keys()

{'value_types'}

The key 'value_types' (notice it's plural) appears in the expected output but not our returned output. Combined with 'value_type' (singular) appearing in the returned output, we can conclude that we forgot an 's' in that key. Seems like we found the problem...

**Not so fast!**

Our solution uses branching logic. We do one thing if the input `some_object` is a `dict` and another thing if it's a `list`. To know where to look in our solution we have to know some more about the `some_object`.

In [ ]:
type(original_input_vars['some_object'])

dict

We're dealing with a `dict` input, so we need to find where we are setting the 'value_type' key and change that to 'value_types'

## Exercise 2 - (**1** Points): 
**Motivation (don't dwell on this)**

In order to maintain server performance and keep response sizes reasonable, the Reddit API paginates results (i.e. it only gives back a slice of the full result). `raw_reddit_data` is actually a `list` of those slices (each slice is a `dict`). Based on the analysis we were able to do with `analyze_structure` in the previous exercise, we can tell that `raw_reddit_data[i]['data']['children']` contains a `list` of all the post data from "page" `i`. We need to combine all of these `lists` into a single `list` to remove some unnecessary complexity from later analysis tasks.

**Requirements**

Define the function `combine_children(raw_reddit_data)`. 

The input `raw_reddit_data` is a `list` of `dict`s such that for any `i` between `0` and `len(raw_reddit_data) - 1`, there will exist a **`list`** `raw_reddit_data[i]['data']['children']`. 

The output should be a *single new* `list` which contains all of the elements of each "children" list *with the order preserved*.

In [ ]:
### Define demo inputs

# child_i_j indicates the j-th element in "list of children" i
demo_raw_reddit_data_ex2 = [
    {'data':{'children':['child_0_0', 'child_0_1']}},
    {'data':{'children':['child_1_0', 'child_1_1']}},
    {'data':{'children':['child_2_0', 'child_2_1']}}
]

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
['child_0_0', 'child_0_1', 'child_1_0', 'child_1_1', 'child_2_0', 'child_2_1']
```
<!-- Include any shout outs here -->
**Note** - the "lists of children" in this example are *greatly simplified* to illustrate the top-level structure of `raw_reddit_data`, the output structure, and the ordering requirement.

In [ ]:
### Exercise 2 solution
def combine_child_data(raw_reddit_data):
    ### BEGIN SOLUTION
    rv = raw_reddit_data[0]['data']['children']
    for page in raw_reddit_data[1:]:
        rv += page['data']['children']
    return rv
    ### END SOLUTION
    
### demo function call
combine_child_data(demo_raw_reddit_data_ex2)

['child_0_0', 'child_0_1', 'child_1_0', 'child_1_1', 'child_2_0', 'child_2_1']

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 2. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [ ]:
### test_cell_ex2
### BEGIN HIDDEN TESTS
import dill
import hashlib
with open('resource/asnlib/public/hash_check.pkl', 'rb') as f:
    hash_check = dill.load(f)
for fname in ['testers.py', '__init__.py', 'test_utils.py']:
    hash_check(f'tester_fw/{fname}', f'resource/asnlib/public/{fname}')
del hash_check
del dill
del hashlib
### END HIDDEN TESTS
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_2', 
    'func': combine_child_data, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'raw_reddit_data':{
            'dtype':'list', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'list',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'S90rT5WLPFy08a82F6SXyiKoyeTi33DOHh7bxXASQw0=', path='resource/asnlib/publicdata/')
for _ in range(20):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

### BEGIN HIDDEN TESTS
tester = Tester(conf, key=b'bE_KdJGq_bhBuoTZRi37O7tu3s38ac4bJUsgUEXnK8s=', path='resource/asnlib/publicdata/encrypted/')
for _ in range(20):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise
### END HIDDEN TESTS
print('Passed! Please submit.')

5e3f0d781ba1b93fcce1788f8448f8ec
5e3f0d781ba1b93fcce1788f8448f8ec
52243648c34549fa35bbbdf59ae56057
52243648c34549fa35bbbdf59ae56057
ed83b476ee8f414c417022217cf79354
ed83b476ee8f414c417022217cf79354


AssertionError: ignored

This time we have a different assertion error. We have modified an input variable! Let's see what it looked like originally and what it looks like after our solution.

In [ ]:
# This prints an insane amount of text. You can uncomment the line below
# if you do not believe me.

# original_input_vars['raw_reddit_data']


Well that's huge. No way we're going to parse that visually. We need to narrow it down a bit.

In [ ]:
type(original_input_vars['raw_reddit_data'])

list

We have a list. Let's do some comparisons with lists. First let's verify that the original and modified lists are the same length...

In [ ]:
len(original_input_vars['raw_reddit_data']) == len(input_vars['raw_reddit_data'])

True

Ok, they're the same length. Now we can iterateively check to see which elements are not the same.

In [ ]:
for i, original_list in enumerate(original_input_vars['raw_reddit_data']):
  modified_list = input_vars['raw_reddit_data'][i]
  if modified_list != original_list:
    print(f'The solution modified raw_reddit_data[{i}]')

The solution modified raw_reddit_data[0]


What type is `raw_reddit_data[0]`?

In [ ]:
type(original_input_vars['raw_reddit_data'][0])

dict

Are the keys the same?

In [ ]:
original_input_vars['raw_reddit_data'][0].keys() == input_vars['raw_reddit_data'][0].keys()

True

Which value is different?

In [ ]:
for key, original_val in original_input_vars['raw_reddit_data'][0].items():
  modified_val = input_vars['raw_reddit_data'][0][key]
  if modified_val != original_val:
    print(f"The solution modified raw_reddit_data[0]['{key}'], which is type {type(original_val)}")

The solution modified raw_reddit_data[0]['data'], which is type <class 'dict'>


Repeat the process for `raw_reddit_data[0]['data']`

In [ ]:
assert original_input_vars['raw_reddit_data'][0]['data'].keys() == input_vars['raw_reddit_data'][0]['data'].keys(), 'key mismatch'
for key, original_val in original_input_vars['raw_reddit_data'][0]['data'].items():
  modified_val = input_vars['raw_reddit_data'][0]['data'][key]
  if modified_val != original_val:
    print(f"The solution modified raw_reddit_data[0]['data']['{key}'], which is type {type(original_val)}")

The solution modified raw_reddit_data[0]['data']['children'], which is type <class 'list'>


Now back to the list strategy!

In [ ]:
len(original_input_vars['raw_reddit_data'][0]['data']['children']) == len(input_vars['raw_reddit_data'][0]['data']['children'])

False

Finally, we have lists of two different lengths! Which one is longer?

In [ ]:
len(input_vars['raw_reddit_data'][0]['data']['children'])

12

In [ ]:
len(original_input_vars['raw_reddit_data'][0]['data']['children'])

4

The `input_vars` one is longer! This means that somewhere we're adding elements to `raw_reddit_data[0]['data']['children']`. Now we need to check our solution and see if we have any references to that which are being modified...

We do! Here's the solution again so you don't have to scroll up.  
```
def combine_child_data(raw_reddit_data):
    ### BEGIN SOLUTION
    rv = raw_reddit_data[0]['data']['children']
    for page in raw_reddit_data[1:]:
        rv += page['data']['children']
    return rv
```
To fix this we could make `rv` a copy of `raw_reddit_data` or (even better) initialize it as an empty list and start our `for` loop at 0.

## Exercise 3a - (**3** Points): 
**Requirements**  

Define the function `clean_text_phase_0(text)` which takes an inputs `text`, a `str` to be cleaned. All of the following transformations must be performed on `text` and the end result returned as a `str`. Some transformations may conflict with others, but using the order provided will give the correct result.

|step number|what to transform|how to identify|transformation|recommendation|  
|---|---|---|---|---|  
|0|all letters|NA|convert to lower case|use `str` methods|  
|1|special unicode characters|handled by `clean_unicode`|Use `clean_unicode`|use provided helper function|  
|2|urls|the sequence '`http`' followed by at least one **non-space character**, i.e. `'the http module'` does not contain a url, but `http://cse6040.gatech.edu` is a url.|remove|use `re` - remove pattern matches|  
|3|space indicators|**space character**, `-`, `+` , or `=`|replace with `' '`|use `re` - replace pattern matches with a single space.|   
|4|posessive ending|`"'s"` (does not have to occur at the end of a **word**)|remove|use `re` - remove pattern matches|  
|5|symbols|`*`, `_`, `:`, `#`, `.`, `,`(the "comma" character), `~`, `?`, `!`, `$`, `%`, `;`, `(`, `)`, `[`, `]`, `/`, `\`, `{`, `}`, `'`, `<`, `>`, `&`, `` ` ``, `"`|remove|use `re` - remove pattern matches. **All symbbols are in demo sample data**.|  
|6|consecutive spaces|consecutive spaces. ex: `'     '`|replace with single space|use `str` methods to split into "list of words", re-combine with `str` methods  

The term "**space character**" should be taken to mean any character which matches the regex pattern `'\s'`. This includes *but is not limited* to spaces, newlines, and tabs. The term "**non-space character**" should be taken to mean all other characters.

The term "**word**" should be taken to mean a series of non-space characters which are either wrapped by either space characters or the start or end of a string. We would interpret the string `'these are all words'` to have 4 words.

## Helper Function `clean_unicode`

Reddit posts allow for unicode special characters and html special characters (letters with accents, emojis, etc.). These characters will cause problems with our analysis. We have provided `clean_unicode(text)` which takes a string input, `text` and returns that string with any such characters replaced with an empty string `''`. You will have to use it in the solution to the next exercise (or figure out a way to strip them out yourself...)

In [ ]:
def clean_unicode(text):
    text = re.sub(r'&.*?;', '', text)
    encoded = text.encode('unicode_escape')
    return re.sub(br"\\u....|\\U........|\\x..", b"", encoded).decode("unicode-escape")

In [ ]:
### Define demo inputs
with open('resource/asnlib/publicdata/children.json') as f:
    children = json.load(f)

demo_text_ex3a = '''
😀 I just love #Python. You will learn to love it, too!!! (Just as much as I do). 


Check out my site at https://cse6040.gatech.edu. It's not much ~~ but I think it's gr8!
        	       It's really cool & fun! The "bee's knees" if you will. 🐝

The request module's functionality really makes HTTP requests a breeze!
allon*_:#.,~?!$%;()\[\]\'/{}\\\'<>&`"eword

"within" should remain, but "it" should not
'''
demo_text_ex3a

'\n😀 I just love #Python. You will learn to love it, too!!! (Just as much as I do). \n\n\nCheck out my site at https://cse6040.gatech.edu. It\'s not much ~~ but I think it\'s gr8!\n        \t       It\'s really cool & fun! The "bee\'s knees" if you will. 🐝\n\nThe request module\'s functionality really makes HTTP requests a breeze!\nallon*_:#.,~?!$%;()\\[\\]\'/{}\\\'<>&`"eword\n\n"within" should remain, but "it" should not\n'

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
'i just love python you will learn to love it too just as much as i do check out my site at it not much but i think it gr8 it really cool fun the bee knees if you will the request module functionality really makes http requests a breeze alloneword within should remain but it should not'
```
<!-- Include any shout outs here -->

**Note** This is a simplified example which admittedly includes some nonsense. It checks most **but not all** requirements. If you need help comparing your output and the expected output you can use <https://text-compare.com/>.

In [ ]:
### Exercise 3a solution
def clean_text_phase_0(text):
    ### BEGIN SOLUTION
    clean = text.lower() # Convert to lowercase
    clean = clean_unicode(clean)
    clean = re.sub(r'http[^\s]+', '', clean) # Remove urls
    clean = re.sub(r'[\s\-=]', ' ', clean) # Replace any space character or dash with ' '
    clean = re.sub(r'\'s', '', clean) # Remove "'s"
    clean = re.sub(r'[*_:#.,~?!$%;()\[\]\'/{}\\\'<>&`"]', '', clean) # Remove punctuation and grouping symbols 
    clean = ' '.join(clean.split())
    ## Moving to another solution
    #     clean = ' '.join(s for s in clean.split() if (re.search(r'[0-9]', s) is None and s not in stopwords)) # remove any "words" (sequences without spaces) containing numbers
    return clean 
    ### END SOLUTION
    
### demo function call
clean_text_phase_0(demo_text_ex3a)

'i just love python you will learn to love it too just as much as i do check out my site at it not much but i think it gr8 it really cool fun the bee knees if you will the request module functionality really makes http requests a breeze alloneword within should remain but it should not'

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 3. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

Depending on the test cases drawn in this cell, a `DepricationWarning` may be raised. **This is nothing to be concerned about**. It indicates that a feature we are using *may* be disabled in a *future* release, but it **will work** for now.

In [ ]:
### test_cell_ex3a
### BEGIN HIDDEN TESTS
import dill
import hashlib
with open('resource/asnlib/public/hash_check.pkl', 'rb') as f:
    hash_check = dill.load(f)
for fname in ['testers.py', '__init__.py', 'test_utils.py']:
    hash_check(f'tester_fw/{fname}', f'resource/asnlib/public/{fname}')
del hash_check
del dill
del hashlib
### END HIDDEN TESTS
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_3a', 
    'func': clean_text_phase_0, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'text':{
            'dtype':'str', # data type of param.
            'check_modified':True,
        },
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'str',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'S90rT5WLPFy08a82F6SXyiKoyeTi33DOHh7bxXASQw0=', path='resource/asnlib/publicdata/')
for _ in range(100):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

### BEGIN HIDDEN TESTS
tester = Tester(conf, key=b'bE_KdJGq_bhBuoTZRi37O7tu3s38ac4bJUsgUEXnK8s=', path='resource/asnlib/publicdata/encrypted/')
for _ in range(100):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise
### END HIDDEN TESTS
print('Passed! Please submit.')

5e3f0d781ba1b93fcce1788f8448f8ec
5e3f0d781ba1b93fcce1788f8448f8ec
52243648c34549fa35bbbdf59ae56057
52243648c34549fa35bbbdf59ae56057
ed83b476ee8f414c417022217cf79354
ed83b476ee8f414c417022217cf79354


AssertionError: ignored

Another error. Our output is not correct again! Let's check the types again.

In [ ]:
assert isinstance(returned_output_vars['output_0'], type(true_output_vars['output_0'])), 'type mismatch'
type(true_output_vars['output_0'])

str

We have strings. Let's print them and compare.

In [ ]:
returned_output_vars['output_0']

'hello ive just released my book on python regular expressions and it is free to download till 13 jan 2019 you can still pay if you wish e book link you will continue to get free updates in future youll need to opt in to get notification via e mail github repo this book would help you learn regular expressions step by step with 200+ examples from basics to advanced levels in addition to re module that comes with standard library the 3rd party regex module is covered as well exercises are also included to test your understanding table of contents 1 preface 2 why is it needed 3 regular expression modules 4 anchors 5 alternation and grouping 6 escaping metacharacters 7 dot metacharacter and quantifiers 8 working with matched portions 9 character class 10 groupings and backreferences 11 lookarounds 12 flags 13 unicode 14 miscellaneous 15 gotchas 16 further reading hope you find the book useful i would be grateful for your feedback and suggestions happy learning'

In [ ]:
true_output_vars['output_0']

'hello ive just released my book on python regular expressions and it is free to download till 13 jan 2019 you can still pay if you wish e book link you will continue to get free updates in future youll need to opt in to get notification via e mail github repo this book would help you learn regular expressions step by step with 200 examples from basics to advanced levels in addition to re module that comes with standard library the 3rd party regex module is covered as well exercises are also included to test your understanding table of contents 1 preface 2 why is it needed 3 regular expression modules 4 anchors 5 alternation and grouping 6 escaping metacharacters 7 dot metacharacter and quantifiers 8 working with matched portions 9 character class 10 groupings and backreferences 11 lookarounds 12 flags 13 unicode 14 miscellaneous 15 gotchas 16 further reading hope you find the book useful i would be grateful for your feedback and suggestions happy learning'

At a glance these look the same. Look a little close and the returned version might be longer? There's got to be a better way to compare strings than just looking at them! Try the tool linked in the propmt, <https://text-compare.com/>! 

We can see from the diff checker that the returned version has "+" which is not expected. Wasn't that one of the things we were supposed to deal with?

## Cosine Similarity
**Read this if you want more information on how we are calculating similarity. All formulas will be given in the exercises where they are used.**

The dictionaries we created for each post's text in the earlier exercises can also be thought of as vectors in a high dimensional space. In this context we can say that there is an angle $\theta \in [0, \pi/2]$ between any two of these vectors $v_0$ and $v_1$. The more similar the two posts are, the smaller the angle will be, thus the larger $\cos\theta$ will be. We can calculate $\cos\theta$ with the following formula.  
$$\cos\theta = \frac{v_0 \cdot v_1}{\|v_0\|\|v_1\|}$$
Let's denote the *unit vectors* with the same direction as $v_0$ and $v_1$ by $\hat{v_0}$ and $\hat{v_1}$ respectively. Then the formula becomes simpler.  
  
$$\cos\theta = \hat{v_0} \cdot \hat{v_1}$$
We can calculate any unit vector $\hat{v_i} \in \mathcal{R}^n$ with the following:
$$\hat{v_i} = \frac{v_i}{\|v_i\|}$$
where
$$\|v_i\| = \sqrt{\sum_{j = 0}^{n-1}{v_{i,j}^2}}$$


## Exercise 7 - (**1** Points): 
**Motivation (don't dwell on this)**

Now that we have normalized word vectors for each post, it's time to implement the similarity calculation.

**Requirements**

Define `calc_similarity(vector_0, vector_1)`. The inputs `vector_0` and `vector_1` will be `dict`s mapping `str` to `float`. Perform the following calculations and return the end result.
- Calculate the product of `vector_0[k]` times `vector_1[k]` for each key `k` that is present in **both** `vector_0` and `vector_1`.
- Compute the sum of all these products **and round to 5 decimal places**.
- Return the **rounded** sum as a `float`.

In [ ]:
### Define demo inputs
with open('resource/asnlib/publicdata/normalized_children.json') as f:
    normalized_children = json.load(f)
demo_vector_0_ex7 = {'python': 0.8, 'jupyter': 0.6}
demo_vector_1_ex7 = {'python': 0.38461538461538464, 'code': 0.9230769230769231}

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
0.30769
1.0
```
<!-- Include any shout outs here -->
**Note** This simplified example contains two runs of `normalize_vector`. The first run uses both demo inputs. The second run uses `demo_vector_0_ex7` as both `vector_0` and `vector_1` to illustrate that comparing a vector to itself should always give the result of `1.0` since $cos(0) = 1$.

In [128]:
### Exercise 7 solution
def calc_similarity(vector_0, vector_1):
    ### BEGIN SOLUTION
    common_keys = vector_0.keys() & vector_1.keys()
    def round_prod(key):
        p = vector_0[key]*vector_1[key]
        return round(p, 5)
    sim = sum(round_prod(key) for key in common_keys)
    return sim
    ### END SOLUTION
    
### demo function call
print(calc_similarity(demo_vector_0_ex7, demo_vector_1_ex7))
print(calc_similarity(demo_vector_0_ex7, demo_vector_0_ex7))

0.30769
1.0


<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 7. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [129]:
### test_cell_ex7
### BEGIN HIDDEN TESTS
import dill
import hashlib
with open('resource/asnlib/public/hash_check.pkl', 'rb') as f:
    hash_check = dill.load(f)
for fname in ['testers.py', '__init__.py', 'test_utils.py']:
    hash_check(f'tester_fw/{fname}', f'resource/asnlib/public/{fname}')
del hash_check
del dill
del hashlib
### END HIDDEN TESTS
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_7', 
    'func': calc_similarity, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'vector_0':{
            'dtype':'dict', # data type of param.
            'check_modified':True,
        },
        'vector_1':{
            'dtype':'dict', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'S90rT5WLPFy08a82F6SXyiKoyeTi33DOHh7bxXASQw0=', path='resource/asnlib/publicdata/')
for _ in range(20):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

### BEGIN HIDDEN TESTS
tester = Tester(conf, key=b'bE_KdJGq_bhBuoTZRi37O7tu3s38ac4bJUsgUEXnK8s=', path='resource/asnlib/publicdata/encrypted/')
for _ in range(20):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise
### END HIDDEN TESTS
print('Passed! Please submit.')

5e3f0d781ba1b93fcce1788f8448f8ec
5e3f0d781ba1b93fcce1788f8448f8ec
52243648c34549fa35bbbdf59ae56057
52243648c34549fa35bbbdf59ae56057
ed83b476ee8f414c417022217cf79354
ed83b476ee8f414c417022217cf79354


AssertionError: ignored

More problems! Let's do the type checking exercise again.

In [130]:
assert isinstance(returned_output_vars['output_0'], type(true_output_vars['output_0'])), 'type mismatch'
type(true_output_vars['output_0'])

float

We're dealing with `float`s this time. Let's see what the absolute difference between our answer and the real answer is.

In [131]:
abs(returned_output_vars['output_0'] - true_output_vars['output_0'])

9.999999999999593e-06

That's a pretty small difference, but it's larger than the tolerance. You can see the tolerance in the test cell. Let's take a look at the values themselves.

In [132]:
returned_output_vars['output_0']

0.03039

In [133]:
true_output_vars['output_0']

0.03038

Somehow we're calculating the last digit incorrectly, but only sometimes. We know it's only sometimes because the demo has the right answer. How can we proceed? Let's extract the inputs and do some trial and error.

In [141]:
### Exercise 7 solution
def calc_similarity(vector_0, vector_1):
    ### BEGIN SOLUTION
    print() # let's make a blank line to differentiate the runs
    common_keys = list(vector_0.keys() & vector_1.keys())
    print({k: (vector_0[k], vector_1[k]) for k in common_keys}) # let's see what we're multiplying
    def round_prod(key):
        p = vector_0[key]*vector_1[key]
        return round(p, 5)
    print({key: round_prod(key) for key in common_keys}) # let's see the products we're adding up
    sim = sum(round_prod(key) for key in common_keys)
    return sim
    ### END SOLUTION
    
### demo function call
print(calc_similarity(demo_vector_0_ex7, demo_vector_1_ex7))
print(calc_similarity(demo_vector_0_ex7, demo_vector_0_ex7))
### add this additional check
print(calc_similarity(**original_input_vars)) 


{'python': (0.8, 0.38461538461538464)}
{'python': 0.30769}
0.30769

{'python': (0.8, 0.8), 'jupyter': (0.6, 0.6)}
{'python': 0.64, 'jupyter': 0.36}
1.0

{'one': (0.05006261743217589, 0.13483997249264842), 'every': (0.05006261743217589, 0.06741998624632421), 'automate': (0.05006261743217589, 0.06741998624632421), 'currently': (0.05006261743217589, 0.06741998624632421), 'excel': (0.10012523486435178, 0.06741998624632421), 'first': (0.10012523486435178, 0.06741998624632421)}
{'one': 0.00675, 'every': 0.00338, 'automate': 0.00338, 'currently': 0.00338, 'excel': 0.00675, 'first': 0.00675}
0.03039


We're rounding the products before adding. It didn't matter in the first example in the demo because there's only one product. In the second product there's nothing to round, so it doesn't matter there either. The test cell example has multiple terms and they're all more precise than 5 decimal places, but we rounded them. Maybe that matters? (A closer read of the prompt should inform you that it does!)

## Exercise 8 - (**3** Points): 

**Motivation (don't dwell on this)** 

We are interested in identifying the posts which are most similar to a given post. We have already done all of the math to capture that information, but it's in yet another huge JSON. 

**Requirements**

Define `n_most_similar(sim_matrix, post_id, n)`. The input `sim_matrix` will be a nested `dict` with the same structure as `similarity_matrix` described in the cell above. The input `post_id` will be the id of the post we are interested in and will be a key of `sim_matrix`. The function should do the following:

- Get the "similarity vector" (`sim_matrix[post_id]`).
- Sort the key/value pairs based on the similarity score. It is recommended but not required to have a `list` of `tuples` for this intermediate result. They should be sorted in *descending* order of similarity. If there are ties (2 or more posts with the same similarity to `post_id`) then break the ties by sorting them into alphabetical order.
- Return a `list` of `tuples` of length `n+1` containing `post_id` and the `n` most similar posts. Tuples should be of the form (post id (`str`), similarity score (`float`)) 

**Note** the post associated with `post_id` will always be at the top of the list since it will have a similarity of `1.0` to itself.

In [ ]:
### Define demo inputs

with open('resource/asnlib/publicdata/similarity_matrix.json') as f:
    similarity_matrix = json.load(f)

demo_sim_matrix_ex8 = {
  'post_0': {'post_0': 1.0,    'post_1': 0.79686,'post_2': 0.67635,'post_3': 0.52459,'post_4': 0.2334},
  'post_1': {'post_0': 0.79686,'post_1': 1.0,    'post_2': 0.65699,'post_3': 0.45758,'post_4': 0.11308},
  'post_2': {'post_0': 0.67635,'post_1': 0.65699,'post_2': 1.0,    'post_3': 0.51968,'post_4': 0.16411},
  'post_3': {'post_0': 0.52459,'post_1': 0.45758,'post_2': 0.51968,'post_3': 1.0,    'post_4': 0.33981},
  'post_4': {'post_0': 0.2334, 'post_1': 0.11308,'post_2': 0.16411,'post_3': 0.33981,'post_4': 1.0}}

demo_n_ex8 = 2
demo_post_id_0 = 'post_0'
demo_post_id_1 = 'post_3'

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
[('post_0', 1.0), ('post_1', 0.79686), ('post_2', 0.67635)]

[('post_3', 1.0), ('post_0', 0.52459), ('post_2', 0.51968)]
```
**Note** This demo calls `n_most_similar` twice. The outputs are separated by a blank line. The first uses `demo_post_id_0` as `post_id`, and the second uses `demo_post_id_1` as `post_id`.

In [142]:
### Exercise 8 solution
def n_most_similar(sim_matrix, post_id, n):
    ### BEGIN SOLUTION
    sims = sorted(sim_matrix[post_id].items(), key=lambda x: -x[1])
    return sims[:n+1]
    ### END SOLUTION
    
print(n_most_similar(demo_sim_matrix_ex8, demo_post_id_0, demo_n_ex8))
print()
print(n_most_similar(demo_sim_matrix_ex8, demo_post_id_1, demo_n_ex8))

[('post_0', 1.0), ('post_1', 0.79686), ('post_2', 0.67635)]

[('post_3', 1.0), ('post_0', 0.52459), ('post_2', 0.51968)]


<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 8. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution. 
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output. 

In [143]:
### test_cell_ex8
### BEGIN HIDDEN TESTS
import dill
import hashlib
with open('resource/asnlib/public/hash_check.pkl', 'rb') as f:
    hash_check = dill.load(f)
for fname in ['testers.py', '__init__.py', 'test_utils.py']:
    hash_check(f'tester_fw/{fname}', f'resource/asnlib/public/{fname}')
del hash_check
del dill
del hashlib
### END HIDDEN TESTS
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_8', 
    'func': n_most_similar, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'sim_matrix':{
            'dtype':'dict', # data type of param.
            'check_modified':True,
        },
        'post_id':{
            'dtype':'str', # data type of param.
            'check_modified':True,
        },
        'n':{
            'dtype':'int', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'S90rT5WLPFy08a82F6SXyiKoyeTi33DOHh7bxXASQw0=', path='resource/asnlib/publicdata/')
for _ in range(100):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

### BEGIN HIDDEN TESTS
tester = Tester(conf, key=b'bE_KdJGq_bhBuoTZRi37O7tu3s38ac4bJUsgUEXnK8s=', path='resource/asnlib/publicdata/encrypted/')
for _ in range(100):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise
### END HIDDEN TESTS
print('Passed! Please submit.')

5e3f0d781ba1b93fcce1788f8448f8ec
5e3f0d781ba1b93fcce1788f8448f8ec
52243648c34549fa35bbbdf59ae56057
52243648c34549fa35bbbdf59ae56057
ed83b476ee8f414c417022217cf79354
ed83b476ee8f414c417022217cf79354


AssertionError: ignored

Yet again we have some faulty code! Debugging this one will be left up to you, although we're going to (time permits) cover it in the associated session.